## Load latest headlines

In [1]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-06-05-04-13-00 +0000,wsj,AUD’s Role as Economic Shock Absorber Might Fade,https://www.wsj.com/finance/currencies/asian-c...
1,2025-06-05-03-49-26 +0000,nyt,Drone Attacks Are the New Front in War. Can NA...,https://www.nytimes.com/2025/06/04/world/europ...
2,2025-06-05-03-22-03 +0000,nyt,Trump Travel Order Bans People From 12 Countri...,https://www.nytimes.com/2025/06/04/us/politics...
3,2025-06-05-03-22-00 +0000,wsj,Judge Blocks Deportation of Boulder Suspect’s ...,https://www.wsj.com/us-news/judge-blocks-depor...
4,2025-06-05-03-04-09 +0000,nyt,U.S. Brings Back Guatemalan Wrongly Deported t...,https://www.nytimes.com/2025/06/04/us/politics...


## Calculate word frequencies

In [2]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words]
counts = Counter(filtered)
score_df = pd.DataFrame(counts.items(), columns=['word','score']).sort_values('score', ascending=False)
score_df.to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-%M-00')
score_df.to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()

C:\Users\CJ\AppData\Local\Temp\ipykernel_19180\1787078784.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-%M-00')


,word,score
1,s,76
17,trump,46
24,u,29
105,south,18
10,new,12


## Rank headlines by score

In [3]:
word_scores = dict(score_df.values)
latest['score'] = latest['title'].apply(lambda t: sum(word_scores.get(w.lower(),0) for w in re.findall(r'[A-Za-z]+', t)))
ranked = latest.sort_values('score', ascending=False)
ranked.to_csv('rank.csv', index=False)
ranked.to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()

,pubdate,source,title,link,score
133,2025-05-30-13-48-00 +0000,wsj,A court ruling that upends many of President T...,https://www.wsj.com/economy/trade/economists-c...,456
78,2025-06-03-17-56-00 +0000,wsj,President Trump’s tariffs threaten to crimp U....,https://www.wsj.com/economy/global/world-growt...,298
117,2025-05-31-15-29-09 +0000,nyt,A Library on the Canada-U.S.Border Is Ensnared...,https://www.nytimes.com/2025/05/30/world/canad...,255
124,2025-05-31-04-08-00 +0000,wsj,International Students Fear Leaving the U.S. a...,https://www.wsj.com/us-news/education/internat...,239
144,2025-05-29-19-51-00 +0000,wsj,President Trump claims tariffs will make the U...,https://www.wsj.com/economy/trade/the-economic...,225


## Next steps